In [9]:
import tkinter as tk

# Tamaño del tablero (8x8)
board_size = 8

# Inicializar los colores de las casillas
white_color = 'white'
black_color = 'black'
default_color = 'lightblue'
board = [[' ' for _ in range(board_size)] for _ in range(board_size)]

# Función para crear una casilla en el tablero
def crear_casilla(tablero, row, col, color=default_color):
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)
    button.grid(row=row + 1, column=col + 1)
    return button

# Función para pintar una casilla
def pintar_casilla(row, col):
    if board[row][col] == ' ':
        color = white_color if current_color == 'W' else black_color
        board[row][col] = current_color
        crear_casilla(tablero, row, col, color)
        cambiar_color()
        imprimir_coordenadas(row, col)  # Imprimir las coordenadas al seleccionar una casilla

# Función para cambiar el color actual
def cambiar_color():
    global current_color
    current_color = 'W' if current_color == 'B' else 'B'
    label_color.config(text=f'Color actual: {current_color}')

def imprimir_coordenadas(row, col):
    print(f"Posición seleccionada: ({row}, {col})")

root = tk.Tk()
root.title('Othello')

tablero = tk.Frame(root)
tablero.pack()

current_color = 'B'

label_color = tk.Label(root, text=f'Color actual: {current_color}')
label_color.pack()

def darw_map():
    for i in range(board_size):
        tk.Label(tablero, text=str(i + 1), width=3).grid(row=i + 1, column=0)
        tk.Label(tablero, text=chr(ord('A') + i), width=3).grid(row=0, column=i + 1)

    # Crear los botones del tablero
    for row in range(board_size):
        for col in range(board_size):
            if (row == 3 and col == 3) or (row == 4 and col == 4):
                crear_casilla(tablero, row, col, 'white')
            elif (row == 3 and col == 4) or (row == 4 and col == 3):
                crear_casilla(tablero, row, col, 'black')
            else:
                crear_casilla(tablero, row, col)

darw_map()
root.mainloop()


Posición seleccionada: (6, 4)
Posición seleccionada: (6, 3)
